<a href="https://colab.research.google.com/github/cdrowley/intro-stat-learning/blob/main/reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Notes / Code translation (from stata) of the following course:
- linkedin.com/learning/11-useful-tips-for-regression-analysis

In [303]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.othermod.betareg import BetaModel
from statsmodels.stats.outliers_influence import variance_inflation_factor


import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [263]:
census = pd.read_stata('../data/census.dta')
census.head(1)

,state,state2,region,pop,poplt5,pop5_17,pop18p,pop65p,popurban,medage,death,marriage,divorce,births
0,Alabama,AL,South,3893888,296412,865836,2731640,440015,2337713,29.299999,35305,49018,26745,62370


In [264]:
auto = pd.read_stata('../data/auto.dta')
auto.head(1)

,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
0,AMC Concord,4099,22,3.0,2.5,11,2930,186,40,121,3.58,Domestic


In [265]:
_401k = pd.read_stata('../data/401k.dta')
_401k.head(1)

,partic,totemp,employ,mrate,prate,age,sole,ltotemp,agesq,ltotempsq
0,188.0,353.0,296.0,0.580821,0.635135,2,not only plan,5.866468,4,34.415447


## Regression Reminders

#### Assumptions of Linear Regression

**1. Linearity:** The relationship between the independent and dependent variables is linear. If it's nonlinear, predictions might be inaccurate.

**2. Independence:** Residuals (differences between observed and predicted values) are independent. If not, predictions could be biased.

**3. Homoscedasticity:** The variance of errors is constant across all independent variable levels. If not, determining the true relationship between variables might be difficult.

**4. Normality:** Errors are normally distributed. If not, predictions may not be reliable.

**5. Absence of Multicollinearity:** Independent variables aren't strongly correlated. Strong correlation or multicollinearity can cause redundancy, falsely impacting the predicted variable.

**6. Lack of Outliers:** The model assumes the absence of outliers. Outliers can significantly skew results and lead to inaccurate predictions.


#### Model Parameters

**Intercept:** Represents the base outcome of the model when all variables are set to 0. It's the 'b' in the linear equation 'y = mx + b'.

**Dependent Variables:** Expected to be continuous numerical values. For string data, each string is separated into categories for individual analysis.

**Coefficient:** Shows how much the independent variable changes with a unit change in the dependent variable. If the coefficient is negative, the variables are inversely related.


#### Statistical Metrics

**R-squared:** Measures how much of the independent variable is explained by changes in our dependent variables.

**Adjusted R-squared:** Important for analyzing multiple dependent variables’ efficacy on the model. Penalizes the R-squared formula based on the number of variables, thus indicating whether some variables may not be contributing properly.

**Df Residuals:** Degrees of freedom, calculated as n-k-1 (number of observations minus number of predicting variables minus 1).

**Covariance:** A measure of how two variables are linked in a positive or negative manner. Robust covariance is calculated to minimize or eliminate variables.

**F-statistic:** In linear regression, it compares your produced model for your variables against a model that nullifies your variables' effects, indicating the statistical significance of your variables.

**Prob (F-Statistic):** Uses the F-statistic to assess the accuracy of the null hypothesis, i.e., whether your variables' effect is indeed zero.

**Standard Error and t-statistic:** The standard error estimates the standard deviation of the coefficient. The t-statistic measures the precision of this estimation, indicating the significance of the coefficient.

**P>|t|:** Represents the probability of your coefficient being measured by chance. It's derived from the t-statistic.

**[0.025 and 0.975]:** These are the values of the coefficients within 95% of the data or within two standard deviations.

**Omnibus and Prob(Omnibus):** Omnibus measures the normality of residuals distribution using skewness and kurtosis. Prob(Omnibus) is the probability that residuals are normally distributed.

**Durbin-Watson:** Measures homoscedasticity or even distribution of errors throughout the data. Ideal homoscedasticity lies between 1 and 2.

**Jarque-Bera (JB) and Prob(JB):** Alternative methods to Omnibus and Prob(Omnibus) that measure normality using skewness and kurtosis.

**Condition Number:** Indicates the sensitivity of the model to changes in the data. High condition number implies multicollinearity.

#### Model Comparison and Selection

**Log-likelihood:** A numerical indicator of the likelihood that your produced model generated the given data. It's used to compare coefficient values for each variable when creating the model.

**AIC and BIC:** Both used to compare the efficacy of models during linear regression, using a penalty system for measuring multiple variables. These numbers are used for feature selection of variables.

## Regression Tips

### Weighted Regression

Run a regression to attempt to explain the number of births a state has by the median age, num marriages, and num divorces

- `marriage` and `divorce` are statistically significant (`medage` is not)
- (Ignoring that) a 1 year increase in a states `medage` decreases the births in a state by ~570 (coefficient), if holding the other variables constant
- different states have different population sizes, so we should weight the regression on `pop`


In [269]:
y = census['births']
x = census[['medage', 'marriage', 'divorce']]

# OLS (with no constant term), so line will start at the origin (0,0) and best fit found only by adjusting slope.
simple_res = sm.OLS(y, x).fit()

# Generally include a constant for better flexibility and predictions.
# Unless there is a good theoretical reason to pass through the origin.
# For instance, when we are certain that the absence of the independent variable(s) should result in a zero outcome for the dependent variable.
# However, ignoring the intercept can lead to biased estimates, as it forces the regression line to go through the origin, which might not be the best fit.
res = sm.OLS(y, sm.add_constant(x)).fit()

In [267]:
# use Patsy's formula language (similar to R)
formula = 'births ~ medage + marriage + divorce'
data = census
weights = census['pop']

m = sm.WLS.from_formula(formula=formula, data=data, weights=weights).fit()
print(m.summary())

                            WLS Regression Results                            
Dep. Variable:                 births   R-squared:                       0.970
Model:                            WLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     489.2
Date:                Sat, 05 Aug 2023   Prob (F-statistic):           6.87e-35
Time:                        20:09:37   Log-Likelihood:                -577.46
No. Observations:                  50   AIC:                             1163.
Df Residuals:                      46   BIC:                             1171.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.212e+04   5.34e+04      0.414      0.6

In [268]:
census.plot(kind='scatter', x='medage', y='births', size='pop', trendline='ols')

### Factor Variables

- Not all regression variables are measured continuosly
- Some variables come in categorical (factor) form (gender, Likert scales, ...)
- To use such variables properly:
  1. code each catefory to a dummy variable
  2. choose a reference (base) category

In the regression, include all but one of the dummy variales. The excluded dummy as as the reference category and all dummy coefficients are relative to it.

Changes to the reference category will lead to changes in the coefficients, but the model remains the same.

In [224]:
# manually handle dummies
dummy = pd.get_dummies(auto['rep78'], prefix='dummy', drop_first=True)
dummy.columns = [c.replace('.0', '') for c in dummy.columns]
auto_w_dummies = pd.concat([auto, dummy], axis=1)
# print(sm.OLS.from_formula(formula='price ~ mpg + dummy_2 + dummy_3 + dummy_4 + dummy_5', data=auto_w_dummies).fit().summary())

In [271]:
# use formula / C() instread
results = sm.OLS.from_formula(formula='price ~ mpg + C(rep78)', data=auto).fit()

categories = auto['rep78'].sort_values().unique()

all_data = pd.DataFrame()
for cat in categories:
    at_mpg = pd.DataFrame({'mpg': np.arange(12, 42), 'rep78': cat})
    predictions = results.get_prediction(at_mpg)
    summ = predictions.summary_frame()

    all_data = pd.concat([all_data, at_mpg.join(summ['mean'])])

In [272]:
fig = px.scatter(
    all_data.dropna().assign(rep78=lambda df: df['rep78'].astype(str))
    , x='mpg'
    , y='mean'
    , color='rep78'
    , color_discrete_sequence=px.colors.qualitative.Plotly
    # , title='Predicted Price by MPG and rep78'
    , labels={'mpg': 'MPG', 'mean': 'Predicted Price'}
    ,
)

fig.for_each_trace( lambda t: t.update(name=f'rep78 = {t.name}') )
fig.update_traces(mode='markers+lines')
fig.show()

### Polynomial Variables

- Polynomial terms in models capture complex, non-linear relationships.
- We need to test if polynomial terms significantly improve our model.
- High-order polynomials can cause multicollinearity, meaning variables predict each other.
- Overfitting, creating a too complex model, can be an issue. Real-life relationships aren't often cubic or higher-order.

In [273]:
# Cars with higher mpg often cost less, even when they're the same length
# a non-linear relationship with miles-per-gallon produces a better model
formulas = {
    'linear': 'price ~ mpg + length'
    , 'quadratic': 'price ~ mpg + np.power(mpg, 2) + length'
    , 'cubic': 'price ~ mpg + np.power(mpg, 3) + length'
}

model = sm.OLS.from_formula(formula=formulas['quadratic'], data=auto)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     12.14
Date:                Sat, 05 Aug 2023   Prob (F-statistic):           1.74e-06
Time:                        20:16:07   Log-Likelihood:                -680.22
No. Observations:                  74   AIC:                             1368.
Df Residuals:                      70   BIC:                             1378.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         2.635e+04   8054.338  

In [274]:
at_mpg_length = pd.DataFrame({
    'mpg': np.linspace(auto['mpg'].min(), auto['mpg'].max(), 100)
    , 'length': auto['length'].mean()
    })

predictions = results.get_prediction(at_mpg_length)
summary_frame = predictions.summary_frame()
fig = go.Figure()

# line for the mean prediction
fig.add_trace(go.Scatter(x=at_mpg_length['mpg'], y=summary_frame['mean'], mode='lines', name='Predicted Price'))

# lines for the confidence interval
fig.add_trace(go.Scatter(x=at_mpg_length['mpg'], y=summary_frame['mean_ci_lower'], mode='lines', name='Lower 95% CI', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=at_mpg_length['mpg'], y=summary_frame['mean_ci_upper'], mode='lines', name='Upper 95% CI', line=dict(dash='dash')))

fig.update_layout(
    title='Marginal Effects of MPG on Price with 95% Confidence Interval'
    , xaxis_title='MPG'
    , yaxis_title='Predicted Price'
)
fig.show()

### Proportional Data

- Voting, occupation rates, the fraction of days that have sunshine
- Proportional data is bounded in the range `[0, 1]` and can take any value including `0 or 1`

#### Linear Regressions with Proportions
- Using LR when the dependent variable is a proportion can result in poor results
- May predict proportions that are above 1 and below 0
- A transformation can ensure predictions are bounded between 0 and 1

#### Solutions
- Beta Regression is very flexible and can model many different data shapes between 0 and 1 (but will not output 0 or 1)
- If 0 or 1 are valid values, then fractional logit and probit regressions are an alternative (less flexible than beta so only use if 0 or 1 is needed)
- Zero-Inflated Beta regression is another alternative

In [279]:
model = sm.OLS.from_formula(formula='prate ~ mrate + totemp + age + sole', data=_401k)
results = model.fit()
_401k['yhatols'] = results.predict(_401k)

In [280]:
model = sm.GLM.from_formula(
    formula='prate ~ mrate + totemp + age + sole'
    , data=_401k
    , family=sm.families.Binomial(link=sm.families.links.logit())
)
results = model.fit()
_401k['yhatlogit'] = results.predict(_401k)

In [282]:
_401k[['yhatols', 'yhatlogit']].describe().T
# yhatols includes predictions above 1
# TODO add beta regression or zero inflated beta regression

,count,mean,std,min,25%,50%,75%,max
yhatols,4075.0,0.840607,0.069427,0.515551,0.792452,0.825444,0.877403,1.126755
yhatlogit,4075.0,0.840607,0.072225,0.397957,0.791525,0.837119,0.894930,0.989903


### Additive Transformations

- Variable transformation is common in regression
- Common transformations are multiplication or division, but we can also add or subtract
- A common method is means centering
- Alternatively, any value can be added or subtracted

#### Why Center a Variable?
1. Improve the interpretation of intercept term (aka value of y when X's = 0), as they are often nonsensical due to being centered on 0 for everything (people aged 0 don't earn a wage, cars with 0 weight don't exists)
2. Reduce collinearity (polynomial terms often lead to increated collinearity


#### Regression Fit Does Not Change
- Centering variables does not change the fit of a regression model
- The underlying model reamins the same, but can help interpretation of intercept coefficients and reduce collinearity

In [299]:
df = auto.copy().assign(
    mpg20 = lambda df: df['mpg'] + 20
    , length200 = lambda df: df['length'] + 200
)

# regression is the same
# but intercept term now refers to a car with 20mpg and length of 200inches (at $5081)
print(sm.OLS.from_formula(formula='price ~ mpg20 + length200', data=df).fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     10.55
Date:                Sat, 05 Aug 2023   Prob (F-statistic):           9.76e-05
Time:                        21:34:08   Log-Likelihood:                -686.09
No. Observations:                  74   AIC:                             1378.
Df Residuals:                      71   BIC:                             1385.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5081.1398   1.19e+04      0.426      0.6

In [318]:
# to replicate stat c.length##c.length
df['length_squared'] = df['length']**2

exog = sm.add_constant(df[['mpg', 'length', 'length_squared']])
endog = df['price']

model = sm.OLS(endog, exog)
results = model.fit()

# Calculate VIF
vif = pd.DataFrame()
vif["variables"] = exog.columns
vif["VIF"] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
vif.set_index('variables').T

variables,const,mpg,length,length_squared
VIF,457.386556,2.735382,4.02084,2.470768


In [319]:
# This model will have the same r-squared + predictions are the same
# but less collinearity

# to replicate stat c.demeaned_length##c.demeaned_length
df['demeaned_length_squared'] = (df['length'] - df['length'].mean())**2

exog = sm.add_constant(df[['mpg', 'length', 'demeaned_length_squared']])
endog = df['price']

model = sm.OLS(endog, exog)
results = model.fit()

# Calculate VIF
vif = pd.DataFrame()
vif["variables"] = exog.columns
vif["VIF"] = [variance_inflation_factor(exog.values, i) for i in range(exog.shape[1])]
vif.set_index('variables').T

variables,const,mpg,length,demeaned_length_squared
VIF,372.753355,2.76985,2.74231,1.017441


variables,const,mpg,demeaned_length_squared
VIF,15.078837,1.011673,1.011673


#### Missing Data

- Missing data is everywhere
- Respondents fail to state their income; ages are not reported; firms do not reveal their profit; GDP measures are missing
- Missing data can often be determined by other information (which we may or not collect)

#### Solutions
- Delete data (not preferred)
- Impute data (better)
  - Univariate (impute using mean, mode, median values)
  - Regression (predict missing values based on a new regression model)
  - Category (code all missing data to a new dummy variable)

### Standarised Estimates


#### How to compare effect sizes?
- Regression coefficients have different units of measurement
- We cannot compare across such different units
- Even when units are the same, their dimension may not be comparable

#### Solution
- Standarise regression coefficients
- Can be done by rescaling all variables (including the dependent variable) to have a mean of 0 and SD of 1
- Coefficient interpretation = A unit change in x1 results in a B1 standard deviation change in y

In [320]:
X = sm.add_constant(df[['mpg', 'headroom', 'trunk', 'weight', 'length']])
y = df['price']

model = sm.OLS(y, X)
results = model.fit()

In [324]:
std_X = X.std()
std_y = y.std()

# Standardize the coefficients
beta = results.params / std_X

# Since the constant doesn't have a meaningful beta, remove it
beta = beta.drop('const')

# Standardize the dependent variable
beta *= std_y

# good way to compare effect sizes (but harder to explain)
beta

mpg        -4.371998e+04
headroom   -2.476004e+06
trunk       7.664366e+04
weight      1.677610e+01
length     -1.431644e+04
dtype: float64

### Regression output

In [343]:
formula = 'price ~ mpg + headroom + trunk + weight + length + C(rep78)'
model = sm.OLS.from_formula(formula=formula, data=df)
results = model.fit()

coefficients = results.params

In [344]:
# graph shows the coefficients as points
# and confidence intervals as bars
# the red 0 line shows null effects
# and anything that crosses the red line is stat. insig.
# and how they compare to each other
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=coefficients.index
    , x=coefficients.values
    , mode='markers'
    , marker=dict(size=10, color='black')
    , error_x=dict(
        type='data' # value of error bar given in data coordinates
        , symmetric=True
        , array=results.bse
        , thickness=1.5
        , color='gray'
      )
    , orientation='h'
))

fig.add_shape(
    type='line'
    , x0=0
    , x1=0
    , y0=-0.5 # adjust to fit plot
    , y1=len(coefficients)-0.5 # adjust to fit your plot
    , line=dict(color='Red', width=1)
)

fig.update_yaxes(autorange="reversed")
fig.show()

### Contour Plots

- Contour plots can be used to visualise complex regression results (such as interaction effects)